In [1]:
import numpy as np
import matplotlib.pyplot as plt
import PyQt5
import xarray as xr

In [2]:
%matplotlib qt

In [6]:
ice = xr.open_dataarray('../data/proc/temporary_zice.nc')

In [7]:
ice

<xarray.DataArray 'ice' (eta_rho: 750, xi_rho: 900)>
[675000 values with dtype=float32]
Dimensions without coordinates: eta_rho, xi_rho
Attributes:
    long_name:  roms_["ice"]
    units:      meter

In [51]:

fig,ax=plt.subplots(figsize=(15,10))
im = ax.pcolormesh(ice!=0)
# cursor = mplcursors.cursor(im, hover=True)
pts = plt.ginput(n=-1,timeout=-1)

plt.show()

pts_arr = np.array(np.round(pts))

KeyboardInterrupt: 

In [66]:

fig,ax=plt.subplots(2,figsize=(15,10))
im = ax[0].pcolormesh(ice!=0)
ax[0].plot(pts_arr[:,0],pts_arr[:,1])
im2 = ax[1].pcolormesh(ice,vmin=-50)
ax[1].plot(pts_arr[:,0],pts_arr[:,1])


In [ ]:
pts_ice_da.values

In [73]:
if input('Are you happy with this? (Y/N)?')=='Y':
    print('continuing...')
else:
    STOP

continuing...


In [84]:
# make path
from matplotlib import path

def inpolygon(xq, yq, xv, yv):
    shape = xq.shape
    xq = xq.reshape(-1)
    yq = yq.reshape(-1)
    xv = xv.reshape(-1)
    yv = yv.reshape(-1)
    q = [(xq[i], yq[i]) for i in range(xq.shape[0])]
    p = path.Path([(xv[i], yv[i]) for i in range(xv.shape[0])])
    return p.contains_points(q).reshape(shape)

xis,etas=np.meshgrid(ice.xi_rho.values,ice.eta_rho.values)

IN = inpolygon(xis,etas,pts_arr[:,0],pts_arr[:,1])

In [92]:
plt.figure()
plt.pcolormesh(ice*~IN,vmin=-20,vmax=0)
plt.colorbar()
plt.show()

In [93]:

pts_ice_da = xr.DataArray(pts_arr,name='ice_mask',dims=['pts','two'],attrs={'long_name': 'coordinates of mask in model coordinate space'})

IN_ice_da = xr.DataArray(IN,name='in_bad_ice_mask',dims=['eta_rho','xi_rho'],attrs={'long_name': 'mask of bad ice'})

ds = xr.Dataset({'IN_icemask':IN_ice_da,
                'pts_icemask':pts_ice_da})

ds.to_netcdf('../data/proc/temporary_zice_mask.nc')